In [71]:
import bs4 as bs
import urllib.request
import pandas as pd
#Open the cat web data page
cat_data = urllib.request.urlopen('https://simple.wikipedia.org/wiki/Cat').read()

# Retrieve the questions and answers
tweet_data = pd.read_csv("question_responce.csv")
tweet_data = tweet_data[["question","responce"]]
#Find all the paragraph html from the web page
cat_data_paragraphs  = bs.BeautifulSoup(cat_data,"html.parser").find_all('p')
#Creating the corpus of all the web page paragraphs
cat_text = ''
#Creating lower text corpus of cat paragraphs
for p in cat_data_paragraphs:
    cat_text += p.text.lower()


In [72]:
tweet_data.head()

,question,responce
0,@AmericanAir Erica on the lax team is amazing ...,@115904 We'll be sure to pass along your kind ...
1,@AmericanAir Could you have someone on your la...,@115904 Our apologies for the delay in respond...
2,Ben Tennyson and an American Airlines pilot. 🎃...,"@115905 Aww, that's definitely a future pilot ..."
3,"@AmericanAir Right, but I earned those. I also...",@115906 We're sorry for your frustration.
4,"Thank you, @AmericanAir for playing #ThisIsUs ...",@115909 We're glad you got to kick back and en...


In [73]:
type(tweet_data)

pandas.core.frame.DataFrame

In [74]:
import re
tweet_data['question'] = tweet_data['question'].map(lambda x: re.sub(r'\W+', ' ', x))
tweet_data['responce'] = tweet_data['responce'].map(lambda x: re.sub(r'\W+', ' ', x)).str.replace('\d+', '')



C:\Users\lubot\AppData\Local\Temp\ipykernel_3604\2454954073.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  tweet_data['responce'] = tweet_data['responce'].map(lambda x: re.sub(r'\W+', ' ', x)).str.replace('\d+', '')


In [75]:
import nltk
nltk.download('punkt')
response_sentences = nltk.sent_tokenize(str(tweet_data.responce))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lubot\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [76]:
print(response_sentences)

['0         We ll be sure to pass along your kind words ...\n1         Our apologies for the delay in responding to...\n2         Aww that s definitely a future pilot in the ...\n3                       We re sorry for your frustration \n4         We re glad you got to kick back and enjoy a ...\n                              ...                        \n1847      We know staying connected is important why n...\n1848      We ve capped our fares for nonstop flights a...\n1849      Please give our Baggage team a call at    fo...\n1850      Our apologies for the hold Our Central Bagga...\n1851      We re providing waivers for St Croix Gillian...\nName: responce, Length: 1852, dtype: object']


In [77]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
def chatbot_answer(user_query):
    
    #Append the query to the sentences list
    response_sentences.append(user_query)
    #Create the sentences vector based on the list
    vectorizer = TfidfVectorizer()
    sentences_vectors = vectorizer.fit_transform(response_sentences)
    
    #Measure the cosine similarity and take the second closest index because the first index is the user query
    vector_values = cosine_similarity(sentences_vectors[-1], sentences_vectors)
    answer = response_sentences[vector_values.argsort()[0][-2]]
    #Final check to make sure there are result present. If all the result are 0, means the text input by us are not captured in the corpus
    input_check = vector_values.flatten()
    input_check.sort()
    
    if input_check[-2] == 0:
        return "Please Try again"
    else: 
        return answer

In [87]:
print("Hello, I am the Airplane Chatbot. How can I help you ?")
while(True):
    query = input().lower()
    if query not in ['bye', 'good bye', 'take care']:
        print("Chatbot: ", end="")
        oui = chatbot_answer(query)
        response_sentences.remove(query)
    else:
        print("See You Again")
        break

with open("Output.txt", "w") as text_file:
    text_file.write(oui)


Hello, I am the Airplane Chatbot. How can I help you ?
Chatbot: See You Again


['0         We ll be sure to pass along your kind words ...\n1         Our apologies for the delay in responding to...\n2         Aww that s definitely a future pilot in the ...\n3                       We re sorry for your frustration \n4         We re glad you got to kick back and enjoy a ...\n                              ...                        \n1847      We know staying connected is important why n...\n1848      We ve capped our fares for nonstop flights a...\n1849      Please give our Baggage team a call at    fo...\n1850      Our apologies for the hold Our Central Bagga...\n1851      We re providing waivers for St Croix Gillian...\nName: responce, Length: 1852, dtype: object']
